In [45]:
STATECODES = {
    'AL':'01',
    'AK':'02',
    'AZ':'04',
    'AR':'05',
    'CA':'06',
    'CO':'08',
    'CT':'09',
    'DE':'10',
    'DC':'11',
    'FL':'12',
    'GA':'13',
    'HI':'15',
    'ID':'16',
    'IL':'17',
    'IN':'18',
    'IA':'19',
    'KS':'20',
    'KY':'21',
    'LA':'22',
    'ME':'23',
    'MD':'24',
    'MA':'25',
    'MI':'26',
    'MN':'27',
    'MS':'28',
    'MO':'29',
    'MT':'30',
    'NE':'31',
    'NV':'32',
    'NH':'33',
    'NJ':'34',
    'NM':'35',
    'NY':'36',
    'NC':'37',
    'ND':'38',
    'OH':'39',
    'OK':'40',
    'OR':'41',
    'PA':'42',
    'RI':'44',
    'SC':'45',
    'SD':'46',
    'TN':'47',
    'TX':'48',
    'UT':'49',
    'VT':'50',
    'VA':'51',
    'WA':'53',
    'WV':'54',
    'WI':'55',
    'WY':'56'
}

In [46]:
USLIST = ['United States', 'U.S.', 'U.S', 'US', 'America', 'United States of America']
DCLIST = ['Washington D.C.', 'DC', 'D.C.', 'Washington DC']
PLACELISTS = [USLIST, DCLIST]
def reformat(place):
    for pl in PLACELISTS:
        if place in pl:
            return pl[0]
    return place

In [47]:
import json
from pprint import pprint
from collections import defaultdict 

data = json.load(open('data/indico-places-out.json'))

places = []
for tweet in data:
    if tweet['places']:
        for place in tweet['places']:
            if place['confidence'] > 0.5:
                places.append(place['text'])

places_dict = defaultdict(int)
for p in places:
    p = reformat(p)
    places_dict[p] += 1


In [48]:
from geopy.geocoders import Nominatim
from geopy import geocoders
from geopy.geocoders import Nominatim
geolocator = geocoders.GeoNames(username="kwhitney")

final = defaultdict(int)
for key in places_dict.keys():
    location = geolocator.geocode(key, timeout=None)
    #location = gn.geocode(key, timeout=None)
    if location:
        loc = location.raw
        cId = loc.get('countryId')
        if cId and cId == '6252001':
            state_name = loc['adminCode1']
            if state_name in STATECODES.keys():
                final[state_name] += places_dict[key]

KeyboardInterrupt: 

In [54]:
with open('staterules.json','r') as f:
    rls = json.load(f)
    
newcodes = {v: k for k, v in STATECODES.items()}

new_freq_dict = {}

for k,v in rls.items():
    new_freq_dict[newcodes[k]] = v

print(new_freq_dict)


{'FL': 46, 'DC': 37, 'AL': 23, 'WV': 20, 'VA': 11, 'MS': 2, 'CA': 5, 'MO': 6, 'AZ': 19, 'TX': 28, 'AK': 6, 'HI': 1, 'NY': 5, 'TN': 14, 'NV': 10, 'SC': 6, 'PA': 11, 'IN': 2, 'ME': 1, 'KY': 4, 'ND': 2, 'LA': 5, 'MA': 3, 'NJ': 7, 'IL': 8, 'CT': 3, 'IA': 3, 'OH': 9, 'WI': 8, 'MT': 2, 'GA': 1, 'MI': 6}


In [55]:
import csv
with open('data/state_to_freq.csv', 'w+') as csvfile:
    wrtr = csv.writer(csvfile)
    wrtr.writerow(['state', 'freq'])
    for k,v in new_freq_dict.items():
        wrtr.writerow([k,v])

In [56]:
# Import libraries
import pandas as pd
import folium
import os

# Load the shape of the zone (US states)
# Find the original file here: https://github.com/python-visualization/folium/tree/master/examples/data
# You have to download this file and set the directory where you saved it
state_geo = 'data/us-states.json'
 
# Load the unemployment value of each state
# Find the original file here: https://github.com/python-visualization/folium/tree/master/examples/data
state_unemployment = 'data/state_to_freq.csv'
state_data = pd.read_csv(state_unemployment)
 
# Initialize the map:
m = folium.Map(location=[37, -102], zoom_start=5)
 
# Add the color for the chloropleth:
m.choropleth(
 geo_data=state_geo,
 name='choropleth',
 data=state_data,
 columns=['state', 'freq'],
 key_on='feature.id',
 fill_color='BuPu',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='Times Mentioned'
)
folium.LayerControl().add_to(m)
 
# Save to html
m.save('output/StatesMentioned.html')
